In [1]:
import pandas as pd
import numpy as np

In [9]:
import spacy

In [2]:
truth_seeker = pd.read_csv("Data/truthSeekerData.csv")

In [50]:
truth_seeker['label'] = 1
truth_seeker

,Title,Date,Article,label
0,#OperationRadiation – The Video BANNED From Yo...,"August 25, 2020","Logic Before Authority – June 16, 2020\n—\n#Op...",1
1,James Corbett in Democracy Down,"July 25, 2020","James Corbett – The Corbett Report July 24, 20...",1
2,"Riots, Race, Religion and Revolution – The Tru...","June 9, 2020","E. Michael Jones – YouTube June 8, 2020",1
3,The Story of Influenza. An Important Tale to T...,"May 15, 2020","No One To Vote For – May 14, 2020\nIt is highl...",1
4,Israeli Historian Discovers ‘6 Million’ Holoca...,"May 1, 2020","Eric Striker – The Unz Review April 30, 2020\n...",1
...,...,...,...,...
210,The British government’s covert propaganda cam...,"February 26, 2020","Ian Cobain, Alice Ross – Middle East Eye Feb 1...",1
211,How to Know If America Is Your Enemy,"February 25, 2020","Eric Zuesse – Strategic Review Feb 24, 2020\nU...",1
212,Flashback: Lest We Forget – Hillary Clinton: W...,"February 25, 2020",By Uprooted Palestinians – Global Research Feb...,1
213,Syrian Patriarch: Islamic Nations Must Admit M...,"February 23, 2020","Church Militant – Jules Gomes Feb 20, 2020\nIg...",1


In [41]:
article = truth_seeker.iloc[214]['Article']

In [61]:
type(truth_seeker.iloc[214]['Article'])

str

In [43]:
nlp = spacy.load("en_core_web_sm")

In [51]:
# doc = nlp(article)
# print(doc)


In [66]:
def tokenize(article):
    doc = nlp(str(article))
    token_list = []
    for token in doc:
        custom_token = token.text
        if token.like_url:
            token.tag_ = 'URL'
            custom_token = '_URL_'
        elif token.is_digit:
            custom_token = '_NUM_'
        token_list.append(custom_token)
    return(' '.join(token_list))

In [63]:
# print(tokenize(article))

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
truth_seeker['Article'] = truth_seeker['Article'].apply(tokenize)

In [68]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(truth_seeker['Article'])

In [69]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, truth_seeker['label'], test_size=0.1, random_state=69)

In [71]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

In [72]:
import numpy as np

predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

1.0


In [73]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[22]]
